# Radiomics Features Extraction

In this notebook, we will be extracting radiomics features for prostate lesions on all mpMRI modalities. These lesions have been manually segmented by radiologists, and we will be using separate settings for each feature extractor.

Radiomics is the process of extracting quantitative features from medical imaging data. These features can provide important information about the characteristics of a lesion, and can be used for a variety of purposes, including diagnosis, prognosis, and treatment planning.

Before we begin, it's important to note that the process of extracting radiomics features can be computationally intensive, so please be patient as the scripts in this notebook run.

Now, let's get started by setting up our environment and importing the necessary libraries.



In [1]:
import sys 
import os 
from config import config # For reading the config files

# Setup the notebook
%load_ext autoreload
%autoreload 2

# Add src to path

if os.path.basename(os.getcwd()) != 'ai4ar-radiomics':
    os.chdir('..')

if 'src' not in sys.path:
    sys.path.append('src')

cfg = config(
    ('json', 'config/config.json', True),
    ('json', 'config/config-ext.json', True), 
    ('json', 'config/radiomics-test.json', True), 
    ignore_missing_paths = True
)

In [2]:
import ai4ar # AI4AR Helper package
from extractor_utils import construct_feature_extractor, extract # Extractor utils from src folder

import pandas as pd # For saving the features

## Dataset initialization

In [3]:
dataset = ai4ar.Dataset(cfg['data_dir'])

In [4]:
# Clinical metadata
dataset[dataset.case_ids[0]].clinical_metadata()

,patient_id,patient_age,mri_source,clinical_group,prior_biopsies,dre_result,psa_result,prostate_volume_us,psa_density_us,prostate_volume_mri,...,isup_group,tnm_pathological_t,postoperative_margin,gs_post,isup_grade_post,tnm_pathological_t_post,tnm_pathological_n_post,tnm_pathological_m_post,eau_risk,treatment
0,1,71,DCO,TN,2.0,0,17.2,64.8,0.27,47.9,...,NaN,t0,0.0,NaN,NaN,NaN,NaN,NaN,IR_L,B


In [5]:
# Radiological metadata
dataset[dataset.case_ids[0]].radiological_metadata()

,patient_id,radiologist_id,lesion_id,TBx,dimension,lesion_dim_sequence,lesion_sectors,lesion_sectors_simple,lesion_zones,lesion_originating,...,label_cor,label_hbv,label_sag,label_t2w,label_dce1,label_dce2,label_dce3,label_dce4,label_dce5,label_dce6
0,1,U_OjmNZ8,1,0,16.0,T2W,"AS-left-in-mid,TZa-left-in-mid","AS,TZa","AS,TZ",TZ,...,NaN,lesion_labels/lesion1/hbv/OjmNZ8,NaN,lesion_labels/lesion1/t2w/OjmNZ8,NaN,NaN,NaN,NaN,NaN,NaN
1,1,U_8ZmM76,1,0,29.0,T2W,"AS-right-in-base,TZa-right-in-base,AS-left-in-...","AS,TZa","AS,TZ",TZ,...,NaN,lesion_labels/lesion1/hbv/8ZmM76,NaN,lesion_labels/lesion1/t2w/8ZmM76,NaN,NaN,NaN,NaN,NaN,NaN
2,1,U_OrjnrO,1,0,31.0,T2W,"AS-left-in-apex,AS-left-in-mid,AS-right-in-mid...",AS,AS,TZ,...,NaN,lesion_labels/lesion1/hbv/OrjnrO,NaN,lesion_labels/lesion1/t2w/OrjnrO,NaN,NaN,lesion_labels/lesion1/dce3/OrjnrO,NaN,NaN,NaN


## Feature Extraction

### Construct the extractors

Extractors are created based on the config/radiomics-test.json configuration

In [6]:

# Possible feature class names =  ['firstorder', 'glcm', 'gldm', 'glrlm', 'glszm', 'ngtdm', 'shape', 'shape2D']

extractors = {}

for modality in cfg['radiomics.settings.extractor'].keys():
    extractors[modality] = construct_feature_extractor(cfg['radiomics.settings.extractor'][modality])


### Extract the radiomics feature

Create the jobs (copies of radiological_metadata dataframe) with info about proper mask and image paths

In [7]:

jobs_dfs = {}

# Create jobs for modalities in the dataset with proper extractor and store them in a dictionary of dataframes
for modality in extractors.keys():    
    # Create a dataframe with the jobs for this modality
    jobs_dfs[modality] = dataset.radiological_metadata[['patient_id', 'lesion_id', 'radiologist_id', 'label_'+modality]].copy()
    # Rename the label column to mask_path 
    jobs_dfs[modality].rename(columns={'label_'+modality: 'mask_path'}, inplace=True)
    # Add the data path column
    jobs_dfs[modality]['data_path'] = 'data/'+modality
    
    # Drop rows with no mask
    jobs_dfs[modality].dropna(subset=['mask_path'], inplace=True)
    
    # If dataset is empty, remove it
    if jobs_dfs[modality].empty:
        del jobs_dfs[modality]
        continue


Extract the features and dump the results for each modality to the tmp dir

In [8]:
# Target directory for the features
floc_dir = os.path.join('.', dataset.tmp_dir)

for modality, jobs_df in jobs_dfs.items():
    floc = os.path.join(floc_dir, f'radiomics_{modality}.csv')
    
    if not os.path.exists(floc):
        print(f'Extracting features for {modality}')
        features = extract(dataset, extractors[modality], jobs_df, n_jobs=4)
        print(f'Features for {modality} extracted, saving')
        
        # Save the not none features
        pd.DataFrame([f for f in features if f is not None]).to_csv(floc, index=False)
        
        # Report the number of cases with no features
        print('Failed features')
        print(jobs_df.loc[[f is None for f in features]][['patient_id', 'lesion_id', 'radiologist_id']])
    else:
        print(f'Features for {modality} already extracted, skipping')
        

Features for adc already extracted, skipping
Features for hbv already extracted, skipping
Features for t2w already extracted, skipping
Features for dce3 already extracted, skipping


In [9]:
import pandas as pd
# Visualize features 


pd.read_csv(os.path.join(dataset.tmp_dir, 'radiomics_t2w.csv')).head()

,patient_id,lesion_id,radiologist_id,mask_path,data_path,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,...,original_glcm_Imc2,original_glcm_InverseVariance,original_glcm_JointAverage,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_MCC,original_glcm_MaximumProbability,original_glcm_SumAverage,original_glcm_SumEntropy,original_glcm_SumSquares
0,1,1,U_OjmNZ8,lesion_labels/lesion1/t2w/OjmNZ8,data/t2w,95.0,159.0,3410783.0,2.248859,32.00,...,0.549885,0.475830,5.672836,0.084822,4.116675,0.442936,0.147789,11.345671,2.699190,1.287434
1,1,1,U_8ZmM76,lesion_labels/lesion1/t2w/8ZmM76,data/t2w,83.0,161.0,18951780.0,2.436913,40.00,...,0.374144,0.468826,5.542182,0.060359,4.592517,0.239668,0.117783,11.084365,2.883047,1.617012
2,1,1,U_OrjnrO,lesion_labels/lesion1/t2w/OrjnrO,data/t2w,79.0,164.1,28481031.0,2.572456,44.00,...,0.331395,0.448691,5.505017,0.049960,4.861650,0.202465,0.104634,11.010035,2.987150,1.896145
3,3,1,U_8ZmM76,lesion_labels/lesion1/t2w/8ZmM76,data/t2w,63.0,201.0,2243638.0,3.056317,57.00,...,0.778955,0.369224,5.363399,0.030463,5.458917,0.482515,0.074127,10.726799,3.440320,4.088800
4,3,2,U_8ekGLQ,lesion_labels/lesion2/t2w/8ekGLQ,data/t2w,26.0,153.9,5056339.0,2.897668,67.75,...,0.589371,0.340918,4.101640,0.025578,5.515494,0.359564,0.050298,8.203280,3.370117,3.745566
